In [13]:
from functools import partial

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as tvt
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder, CIFAR100, CIFAR10
from tqdm import tqdm

from compvit.factory import compvit_factory
from compvit.layers.bottleneck import conv_bottleneck
from compvit.layers.compressor import Compressor
from compvit.layers.inverted_bottleneck import inverted_mlp
from dinov2.factory import dinov2_factory
from dinov2.layers import Mlp

In [14]:
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')

Using cache found in /home/jajal/.cache/torch/hub/facebookresearch_dinov2_main


ModuleNotFoundError: No module named 'dinov2.hub'

In [2]:
class CompresserDecoder(nn.Module):
    def __init__(self, compressor, total_tokens, num_compressed_tokens) -> None:
        super().__init__()
        self.compressor = compressor
        self.decoder = nn.Linear(num_compressed_tokens, total_tokens)
    
    def forward(self, x):
        x = self.compressor(x)
        x = self.decoder(x.mT).mT
        return x

# Setup Dataset

In [3]:
def calculate_mean_std(dataset):
    mean = 0
    std = 0
    loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)
    for img, _ in loader:
        batch_mean = torch.mean(img, dim=(0, 2, 3))
        batch_std = torch.std(img, dim=(0, 2, 3))
        mean += batch_mean
        std += batch_std
    mean /= len(loader)
    std /= len(loader)
    return mean, std

In [4]:
mean, std = calculate_mean_std(ImageFolder(root="data/synthetic-checkerboard/train", transform=tvt.ToTensor()))
transform = tvt.Compose([tvt.ToTensor(), tvt.Normalize(mean, std)])
train_dataset = ImageFolder("data/synthetic-checkerboard/train", transform=transform)
val_dataset = ImageFolder("data/synthetic-checkerboard/val", transform=transform)

In [5]:
TRANSFORM = tvt.Compose(
    [
        tvt.RandomCrop(32, padding=4),
        tvt.Resize(224),
        tvt.RandomHorizontalFlip(),
        tvt.ToTensor(),
        tvt.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

TRANSFORM_TEST = tvt.Compose(
    [
        tvt.Resize(224),
        tvt.ToTensor(),
        tvt.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

train_dataset = CIFAR10("toy_experiments/data", transform=TRANSFORM, download=True)
val_dataset = CIFAR10(
    "toy_experiments/data", transform=TRANSFORM_TEST, train=False, download=True
)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=1)

# Evaluate and Training Code

In [7]:
def evaluate_reconstruction(model, compressor_decoder, val_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for img, _ in tqdm(val_loader):
            img = img.to("cuda")
            tokens = model.prepare_tokens_with_masks(img)
            pred_tokens = compressor_decoder(tokens)
            loss = F.mse_loss(pred_tokens, tokens, reduction="mean")
            total_loss += loss.item()
    return total_loss / len(val_loader)

In [8]:
def train(model, compressor_decoder, train_loader, epochs, optimizer):
    for epoch in range(epochs):
        compressor_decoder.train()
        total_loss = 0.0
        for img, _ in tqdm(train_loader):
            img = img.to("cuda")
            with torch.no_grad():
                tokens = model.prepare_tokens_with_masks(img)
            pred_tokens = compressor_decoder(tokens)
            loss = F.mse_loss(pred_tokens, tokens, reduction="mean")
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_loss = loss.item()
        print(f"Epoch {epoch}: {total_loss / len(train_loader):.2e}")
        print(f"Validation loss: {evaluate_reconstruction(model, compressor_decoder, val_loader):.2e}")

# Setup Default Models and Parameters

In [9]:
embed_dim = 384
num_heads = 6
mlp_ratio = 4
qkv_bias = True
ffn_bias = True
proj_bias = True
norm_layer = partial(nn.LayerNorm, eps=1e-6)
act_layer = nn.GELU
ffn_layer = Mlp
init_values = 1.0
num_compressed_tokens = 17
total_tokens = 257

bottleneck_size = 1
bottleneck = partial(
    conv_bottleneck,
    dim=embed_dim,
    ratio=mlp_ratio,
    bottleneck_size=bottleneck_size,
)

inv_bottle_size = 1
codebook_ratio = 2
inv_bottleneck = partial(
    inverted_mlp,
    dim=embed_dim,
    ratio=codebook_ratio,
)

In [10]:
model, cfg = dinov2_factory("dinov2_vits14")
model.load_state_dict(torch.load("dinov2/checkpoints/dinov2_vits14_pretrain.pth"))

<All keys matched successfully>

In [11]:
def create_compressor(num_codebook_tokens):
    compressor = Compressor(
        dim=embed_dim,
        num_heads=num_heads,
        mlp_ratio=mlp_ratio,
        qkv_bias=qkv_bias,
        proj_bias=proj_bias,
        ffn_bias=ffn_bias,
        norm_layer=norm_layer,
        act_layer=act_layer,
        ffn_layer=ffn_layer,
        init_values=init_values,
        num_compressed_tokens=num_compressed_tokens,
        num_tokens=total_tokens,
        bottleneck=bottleneck,
        num_codebook_tokens=num_codebook_tokens,
        inv_bottleneck=inv_bottleneck,
    )
    return compressor

# Setup Up Models -- codebook tokens > total tokens

In [12]:
num_codebook_tokens = 384
compressor = create_compressor(num_codebook_tokens)

In [13]:
model = model.to("cuda")
compressor_decoder = CompresserDecoder(
    compressor,
    total_tokens,
    num_compressed_tokens,
).to("cuda")
total_params = sum(p.numel() for p in compressor_decoder.parameters())
print(f"Number of parameters in CompressorDecoder: {total_params:,}")

Number of parameters in CompressorDecoder: 5,160,476


In [14]:
epochs = 50
optimizer = torch.optim.AdamW(compressor_decoder.parameters(), lr=1e-6, weight_decay=5e-2)

In [15]:
train(model, compressor_decoder, train_loader, epochs, optimizer)
test_loss = evaluate_reconstruction(model, compressor_decoder, val_loader)
print(f"Final test loss: {test_loss:.2e}")

100%|██████████| 782/782 [00:32<00:00, 24.07it/s]


Epoch 0: 3.49e-05


100%|██████████| 313/313 [00:06<00:00, 47.50it/s]


Validation loss: 2.67e-02


100%|██████████| 782/782 [00:32<00:00, 24.12it/s]


...

Epoch 49: 8.22e-06


100%|██████████| 313/313 [00:06<00:00, 47.31it/s]


Validation loss: 6.05e-03


100%|██████████| 313/313 [00:06<00:00, 48.05it/s]

Final test loss: 6.05e-03


# Setup Up Models -- codebook tokens = total tokens

In [16]:
num_codebook_tokens = 257
compressor = create_compressor(num_codebook_tokens)

In [17]:
model = model.to("cuda")
compressor_decoder = CompresserDecoder(
    compressor,
    total_tokens,
    num_compressed_tokens,
).to("cuda")
total_params = sum(p.numel() for p in compressor_decoder.parameters())
print(f"Number of parameters in CompressorDecoder: {total_params:,}")

Number of parameters in CompressorDecoder: 5,160,476


In [18]:
epochs = 50
optimizer = torch.optim.AdamW(compressor_decoder.parameters(), lr=1e-6, weight_decay=5e-2)

In [19]:
train(model, compressor_decoder, train_loader, epochs, optimizer)
test_loss = evaluate_reconstruction(model, compressor_decoder, val_loader)
print(f"Final test loss: {test_loss:.2e}")

100%|██████████| 782/782 [00:29<00:00, 26.55it/s]


Epoch 0: 3.44e-05


100%|██████████| 313/313 [00:06<00:00, 48.27it/s]


Validation loss: 2.64e-02


100%|██████████| 782/782 [00:29<00:00, 26.53it/s]


...

Epoch 49: 8.38e-06


100%|██████████| 313/313 [00:06<00:00, 45.72it/s]


Validation loss: 6.33e-03


100%|██████████| 313/313 [00:06<00:00, 46.75it/s]

Final test loss: 6.33e-03


# Setup Up Models -- codebook tokens < total tokens

In [20]:
num_codebook_tokens = 128
compressor = create_compressor(num_codebook_tokens)

In [21]:
model = model.to("cuda")
compressor_decoder = CompresserDecoder(
    compressor,
    total_tokens,
    num_compressed_tokens,
).to("cuda")
total_params = sum(p.numel() for p in compressor_decoder.parameters())
print(f"Number of parameters in CompressorDecoder: {total_params:,}")


Number of parameters in CompressorDecoder: 5,160,476


In [22]:
epochs = 50
optimizer = torch.optim.AdamW(compressor_decoder.parameters(), lr=1e-6, weight_decay=5e-2)

In [23]:
train(model, compressor_decoder, train_loader, epochs, optimizer)
test_loss = evaluate_reconstruction(model, compressor_decoder, val_loader)
print(f"Final test loss: {test_loss:.2e}")

100%|██████████| 782/782 [00:27<00:00, 28.82it/s]


Epoch 0: 3.40e-05


100%|██████████| 313/313 [00:06<00:00, 45.84it/s]


Validation loss: 2.65e-02


100%|██████████| 782/782 [00:27<00:00, 28.91it/s]


...

Epoch 49: 8.80e-06


100%|██████████| 313/313 [00:06<00:00, 46.26it/s]


Validation loss: 6.55e-03


100%|██████████| 313/313 [00:06<00:00, 46.49it/s]

Final test loss: 6.55e-03
